In [ ]:
# Datos: https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv

In [1]:
import requests
import pandas as pd
from google.cloud import storage

In [2]:
URL = 'https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv'
FILEPATH = './data/camas_bogota_raw.csv'

In [3]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [4]:
camas = pd.read_csv( './data/camas_bogota_raw.csv', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': str, 'Camas habilitadas': str, 'Camas disponibles': str } )

In [5]:
camas[ 'Camas Ocupadas' ] = camas[ 'Camas Ocupadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas habilitadas' ] = camas[ 'Camas habilitadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas disponibles' ] = camas[ 'Camas disponibles' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )

In [6]:
camas.dtypes

IPS                        object
Servicio                   object
Grupo                      object
Camas Ocupadas              int32
Camas habilitadas           int32
Camas disponibles           int32
Porcentaje de Ocupación    object
Fecha Actualización        object
dtype: object

In [7]:
camas

,IPS,Servicio,Grupo,Camas Ocupadas,Camas habilitadas,Camas disponibles,Porcentaje de Ocupación,Fecha Actualización
0,Públicas,Hospitalización General,Neonatal,110,138,28,80%,12/08/2020
1,Públicas,Hospitalización General,Pediátrico,108,319,211,34%,12/08/2020
2,Públicas,Hospitalización General,Adulto,1070,1552,482,69%,12/08/2020
3,Públicas,Unidad de Cuidado Intermedio,Neonatal,98,126,28,78%,12/08/2020
4,Públicas,Unidad de Cuidado Intermedio,Pediátrico,3,8,5,38%,12/08/2020
5,Públicas,Unidad de Cuidado Intermedio,Adulto,28,40,12,70%,12/08/2020
6,Públicas,Unidad de cuidado intensivo,Neonatal,54,68,14,79%,12/08/2020
7,Públicas,Unidad de cuidado intensivo,Pediátrico,14,24,10,58%,12/08/2020
8,Públicas,Unidad de cuidado intensivo,Adulto,375,410,35,91%,12/08/2020
9,Privadas,Hospitalización General,Neonatal,121,162,41,75%,12/08/2020


In [8]:
camas_grouped = camas.groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).reset_index()

In [9]:
camas_grouped[ '% Ocupación' ] = camas_grouped[ 'Camas Ocupadas' ] / camas_grouped[ 'Camas habilitadas' ]

In [10]:
camas_grouped[ 'Servicio' ] = camas_grouped[ 'Servicio' ].replace( { 'Hospitalización General': 'General', 'Unidad de Cuidado Intermedio': 'Intermedio', 'Unidad de cuidado intensivo': 'UCI' } )

In [11]:
camas_grouped.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [12]:
ucis = pd.read_csv( 'https://storage.googleapis.com/siscovid/uci_bogota.csv' )

In [13]:
ucis_num = ucis[ 'Camas UCI ocupadas Covid-19' ].values[ -1 ]

In [14]:
ucis_num

1517

In [15]:
listofzeros = [ 0 ] * 3
listofzeros[ list( camas_grouped[ 'Servicio' ].values ).index( 'UCI' ) ] = ucis_num
listofzeros

[0, 0, 1517]

In [16]:
camas_grouped[ 'Camas Ocupadas COVID' ] = pd.Series( listofzeros )

In [17]:
camas_grouped[ 'Camas Ocupadas' ] = camas_grouped[ 'Camas Ocupadas' ] - camas_grouped[ 'Camas Ocupadas COVID' ]

In [18]:
camas_grouped

,Servicio,Camas Ocupadas,Camas habilitadas,Camas disponibles,Ocupacion,Camas Ocupadas COVID
0,General,6416,8780,2364,0.730752,0
1,Intermedio,549,877,328,0.625998,0
2,UCI,637,2465,311,0.873834,1517


In [19]:
#camas_grouped.to_csv( './data/camas_bogota.csv', index = False )

In [20]:
#DataFrame to CSV
csv_data=camas_grouped.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('camas_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')